In [40]:
import pandas as pd 
import numpy as np
import dill


In [23]:
X_train= pd.read_csv('dataset/X_train_Final.csv')
y_train= pd.read_csv('dataset/y_train.csv')

In [24]:
X_test= pd.read_csv('dataset/X_test_Final.csv')
y_test= pd.read_csv('dataset/y_test.csv')

In [25]:
X_train=X_train.drop("Sale_Price", axis=1)
X_test=X_test.drop("Sale_Price", axis=1)

In [39]:
X_train=X_train.drop("id", axis=1)
X_test=X_test.drop("id", axis=1)

In [50]:
y_train=y_train.squeeze()
y_test=y_test.squeeze()

In [42]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn import model_selection
from sklearn.metrics import mean_squared_error

# RANDOM FOREST REGRESSOR

In [29]:
rf = RandomForestRegressor()
rf.fit(X_train,y_train)

C:\Users\Marco\AppData\Local\Temp\ipykernel_3704\1932820455.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train,y_train)


RandomForestRegressor()

In [35]:
y_prediction= rf.predict(X_test)

In [38]:
mse = mean_squared_error(y_test, y_prediction)
rmse = mse**.5

print(f'RMSE : {rmse}')

RMSE : 23864.92706044465


## FACCIAMO TUNING DEI PARAMETRI DELLA RANDOM FOREST

In [78]:
## Define Grid 
from datetime import datetime


param_grid = {'n_estimators': [500,1000,2000,4000], #numero di alberi nella foresta
              'min_samples_split': [2], #numero di campioni prelevati per addestrare gli alberi
              'bootstrap' : [True],
              'max_depth' : [5,6,7,None],
              #'min_samples_split':2,
              'max_features': ['sqrt','log2', 0.3],
              } #numero di feature su cui sono allenati gli alberi

grid_search = model_selection.GridSearchCV(rf, 
                                           param_grid,
                                           scoring="neg_root_mean_squared_error", #MSE MEAN SQUARE ERROR
                                           verbose=3,
                                           n_jobs=-1, #multi processor
                                           #cv = 5 <-- default cross validation
                                           return_train_score=True)
## show start time
print(datetime.now())
best_model = grid_search.fit(X_train, y_train)#Allena tanti modelli quante sono le combinazioni tra i parametri definiti in param_grid
print('Optimum parameters', best_model.best_params_)
## show end time
print(datetime.now())

2022-12-29 22:39:12.899880
Fitting 5 folds for each of 48 candidates, totalling 240 fits
Optimum parameters {'bootstrap': True, 'max_depth': None, 'max_features': 0.3, 'min_samples_split': 2, 'n_estimators': 2000}
2022-12-29 22:50:10.791188


In [79]:
rf_optimized = RandomForestRegressor(bootstrap=True, max_depth=None, min_samples_split=2, n_estimators=2000, max_features=0.3)
rf_optimized.fit(X_train,y_train)

RandomForestRegressor(max_features=0.3, n_estimators=2000)

In [80]:
y_prediction_optimized= rf_optimized.predict(X_test)

In [81]:
mse_optimized = mean_squared_error(y_test, y_prediction_optimized)
rmse_optimized = mse_optimized**.5
print(f'RMSE : {rmse_optimized}')

RMSE : 22654.765302694206


In [82]:
results = pd.DataFrame({'original': y_test, 'predicted': y_prediction_optimized}, columns=['original', 'predicted'])

In [90]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt

In [87]:

fig =px.scatter(results, x="original", y="predicted",trendline="ols",height=1000, width=1000)
fig.show()

In [83]:
import dill

def save_session() :
    dill.dump_session('data_prediction.db')
    
def load_session():
    dill.load_session('data_prediction.db')

In [84]:
#save_session()